In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from hidmed import *
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.lines import Line2D

## Evaluate estimators

In [ ]:
setups = ["a", "b", "c"]
sample_sizes = [300, 900, 1500]
dims = [1, 5]
num_evals = 100

results = {}

for setup in setups:
    for n in sample_sizes:
        for dim in dims:
            if dim == 1:
                xdim, zdim, wdim, mdim, udim = 1, 1, 1, 1, 1
            else:
                xdim, zdim, wdim, mdim, udim = 5, 2, 2, 2, 1

            # set up model
            datagen = LinearHidMedDGP(xdim, zdim, wdim, mdim, udim, setup=setup, seed=0)
            true_psi = datagen.true_psi()

            # hyperparameter tuning
            dataset = datagen.sample_dataset(n, seed=1)
            tuner = ProximalMultiplyRobust("c", folds=2, num_runs=200, num_jobs=1)
            tuner.fit(dataset)

            # estimation
            for estimator in [
                ProximalMultiplyRobust,
                ProximalInverseProbWeighting,
                ProximalOutcomeRegression,
            ]:
                print(
                    f"Running {estimator.__name__} for {dim}-dimensional case, n={n}, setup={setup}"
                )
                predictor = estimator(setup, folds=2, param_dict=tuner.param_dict)
                dataset = datagen.sample_dataset(n, seed=1)
                predictor.fit(dataset)

                # evaluation
                res = {
                    "estimate": np.zeros(num_evals),
                    "bias": np.zeros(num_evals),
                    "mse": np.zeros(num_evals),
                    "anb": np.zeros(num_evals),
                    "covered": np.zeros(num_evals),
                    "ci_width": np.zeros(num_evals),
                }
                for i in tqdm(range(1, num_evals + 1)):
                    if i == 1:
                        eval_data = None
                    else:
                        eval_data = datagen.sample_dataset(2 * n // 3, seed=i)
                    estimate = predictor.evaluate(
                        eval_data=eval_data, reduce=False, verbose=False
                    )

                    res["estimate"][i - 1] = np.mean(estimate)
                    res["bias"][i - 1] = np.mean(estimate) - true_psi
                    res["mse"][i - 1] = calculate_mse(estimate, true_psi)
                    res["anb"][i - 1] = absolute_normalized_bias(estimate, true_psi)
                    res["covered"][i - 1] = is_covered(estimate, true_psi)
                    res["ci_width"][i - 1] = confidence_interval(estimate)

                results[dim, setup, n, estimator.__name__] = res

pickle.dump(results, open("assets/results.pkl", "wb"))

## Visualize results

In [ ]:
results = pickle.load(open("assets/results.pkl", "rb"))

In [ ]:
dims = 5
estimators = [
    ProximalMultiplyRobust,
    ProximalInverseProbWeighting,
    ProximalOutcomeRegression,
]
for setup in setups:
    idx = 1
    positions = []
    for col in range(len(sample_sizes)):
        for i in range(len(estimators)):
            positions.append(idx)
            idx += 1
        idx += 2

    labels = [
        sample_sizes[col] if i == 2 else " "
        for col in range(len(sample_sizes))
        for i in range(3)
    ]

    fig, ax = plt.subplots(figsize=(3 * 2, 4))
    plotdata = []
    for n in sample_sizes:
        for estimator in [est.__name__ for est in estimators]:
            plotdata.append(np.array(results[dims, setup, n, estimator]["bias"]))

    bplot = ax.boxplot(
        np.vstack(plotdata).T,
        positions=positions,
        labels=labels,
        patch_artist=True,
        widths=1,
        showmeans=True,
    )

    base_colors = ["cornflowerblue", "mediumaquamarine", "khaki"]
    colors = [c for _ in range(len(sample_sizes)) for c in base_colors]
    for patch, color in zip(bplot["boxes"], colors):
        patch.set_facecolor(color)

    custom_lines = [Line2D([0], [0], color=c, lw=4) for c in base_colors]
    ax.legend(custom_lines, ["POR", "PIPW", "PMR"])

    ax.axhline(y=0.0, color="k", linestyle="--", zorder=3)
    # plt.ylim(-2.5, 2.5)
    plt.xticks(positions[1 :: len(estimators)], sample_sizes)
    plt.xlabel("Sample Size")
    plt.ylabel("Bias")
    plt.tight_layout()
    # plt.savefig("assets/prox_hidmed_1d_sample_sizes_setup_%s.png" % mode, dpi=500)
    plt.show()

In [ ]:
for dim in dims:
    for setup in setups:
        for n in sample_sizes:
            # Generate the key for the current combination
            key = (dim, setup, n, "ProximalMultiplyRobust")

            # Check if the key exists in the dictionary
            if key in results:
                # Retrieve the sub-dictionary for the current key
                data = results[key]

                # Print averages for each metric
                print(f"Summary for {key}:")
                for metric in ["estimate", "bias", "mse", "anb", "covered", "ci_width"]:
                    average = np.mean(data[metric])
                    print(f"  Average {metric}: {average:.4f}")
                print()